# High speed signal processing without FPGA's

High speed digitizers can generate samples really fast but often provide only little means to adding a signal processing chain using on-boards FPGA's. Moreover, FPGA programming is HARD and not at all that flexible. It would be nice when we can use our own measurement computers to run the signal processing using either the CPU or perhaps the GPU (through CUDA). In this work we benchmark several options for DSP on a 'normal' workstation computer. 


## Numpy
A easiest approach is Numpy, and in fact it's also one of the fastest approaches provided you install a Numpy version with MKL support. The Numpy provided by the default 'anaconda' channel have MKL as does the version on the 'intel' channel, but the version provided on the 'conda-force' for exmaple does not! In fact I would recommend to install from the anaconda channel, because the one from 'intel' is compiled in a 'weird' way (check out numpy.show_config() and compare).
Numpy+MKL is fast because it uses C and MKL, but it doesn't support multi-threading (without GIL) and it copies data often. This makes it slower than it has to be. The altenative options explored are either enabling multi-threading (CPU or GPU based) and/or use IPP/MKL for processing with zero-copying. 

## Numexpr
The package is intended as a fast evaluator of simple numerical expressions without copying data and by supporting multi-threading. It doesn't a lot of mathematical expressions, but for the few it does, and provided you enable multi-threading, its actually a lot faster than Numpy. This will not be a complete solution, but can be, at least, part of the solution. 

## Numba
This is a very easy approach to run python code at the 'speed of C or FORTRAN'. This is done by translating "Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library." ([link](https://numba.pydata.org/)), without the need on compilers present at the system. Moreover, the LLVM compiler uses SIMD instructions to benefit from the CPU hardwar as much as possible ([link](https://hub.gke2.mybinder.org/user/numba-numba-examples-13t8chqr/notebooks/notebooks/simd.ipynb)). Numba also has easy access support for running function parallel without impedance of the GIL. 
However, the approach comes at the cost of loosing MKL and Numpy/Scipy function (because those won't run) and the requirement to understand how to structure code to actually benefit from the SIMD optiomizations. 

### CUDA
One of the strengths of Numba is the support for GPU hardware ([link](https://numba.readthedocs.io/en/stable/cuda/overview.html)) i.e. one can run 'python' code on the GPU without a real need to learn or understand CUDA at all. Depending on the CUDA hardware this could perhaps turn out to be a very fast solutions. In fact there exists a couple of packages that provided CUDA kernels for quite a big portion of the Numpy and Scipy packages, so one would need to write even less code than using Numbda using the CPU. 

> #### Cupy
The package implements most of the functionality of the Numpy package but runs its at the GPU. 

> #### CUSIGNAL
The package implements most of the functionality of the Scipy.signal package but runs its at the GPU. 

It appears that Numba would be a great solution when we are considering using the GPU but it's usefulness for CPU code has to be seen (mostly because Numpy+MKL is that fast already).

Material:
*   Tutorial ([link](https://nyu-cds.github.io/python-numba/05-cuda/))
*   An introduction into CUDA [link](https://www.vincent-lunot.com/post/an-introduction-to-cuda-in-python-part-1/)
*   Some benchmark (not that usefull) ([link](https://towardsdatascience.com/python-performance-and-gpus-1be860ffd58d))

## Pythran
This package is a bit similar to Numba but implements an 'ahead-of-time' compilation strategy i.e. it compiles Python code to C modules to be used by Python. It supports multi-threading (via OpenMP instructions) and it is supposed to support SIMD instructions (but I never got it to actually work). 

## IPP (+OpenMP)
Numpy+MKL is fast because it uses Intel MKL to run the calculations. There exists another package, Intel IPP that implements roughly the same functionality but appears to be a bit faster when the data to process is 'small'. To use this Python needs to interface with the C code which can be doen in many ways. There are several works available that outline the different possibilities ([link](https://realpython.com/python-bindings-overview/), [link](https://scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html), [link](https://docs.python-guide.org/scenarios/clibs/), [link](https://cai.tools.sap/blog/how-to-speed-up-python-with-c/)) Among all possibilities we selected those that seem easy to implement and/or we already use (e.g. ctypes).

> #### ctypes (no multi-threading)
This the the de-facto way (because it's part of python) of interfacing with C code on a per-function call basis. But calling C function with ctypes has quite some overhead because it needs to translate Python object to C object and back. So, its not really a solution for high speed processing, but helpfull for getting started with an external library. 

> #### CFFI (no multi-threading)
A easy and faster way to interface with C code on a per-function call basis. It's fast but a bit clunky...

> #### cython (with OpenMP multi-threading)
This is much more thanjust a library to interface with C code. In fact, cython allows one to write modules for python in C in a very easy way. Moreover, it can link against external C and C++ libraries, supports OpenMP and can talk with Numpy without the need of copying data. This makes it a very good candidate for our high speed signap processing back-end. Writing cython is not difficult, but perhaps a big too much when you don't care about programming. Therefor we should implement default out-of-the-box functionality as well as some modular support which integrates well with Numpy/Scipy and Numexpr. 

# Setting up the environment
Here is a list of actions required to get your environment set up.
*   Install Anaconda and create a new environment to contain all experiments (I used Python 3.8). 
*   Install the MSVC compiler compatible with you python version ([link](https://wiki.python.org/moin/WindowsCompilers), [link](https://visualstudio.microsoft.com/downloads/#build-tools-for-visual-studio-2019))

*   Install the Intel C compiler + MKL + IPP ([link](https://dynamicinstaller.intel.com/system-studio/))
*   Install The CUDA Toolkit (tested with 11 update 1) ([link](https://developer.nvidia.com/cuda-downloads))

###Packages
*   **Numpy** (anaconda channel): conda install -c anaconda numpy
*   **Scipy**, **matplotlib**, **ipykernel**, **spyder**, etc
*   **cython**: conda install cython
*   **Numba**: conda install numba 
*   **cudatoolkit**: conda install cudatoolkit
*   **cupy**: pip install cupy-cuda110
*   **cuSignal**: download ([link](https://github.com/rapidsai/cusignal/tree/branch-0.16)) and install (python setup.py install)
*   **IPP dlls**: conda install -c intel ipp 

    Installs Intel IPP dlls under '<envname>\Library\bin' required to run IPP based (C) code.


# Numexpr specific instructions
You need the MKL package from Intel (you don't need the Fortran compiler). 
Makes sure you have the numpy from the anaconda channel. This is because the anaconda channel offers numpy compiled with the mkl-rt library. You can check this with numpy.show_config().

*   You need to download the numexpr source. 
*   Rename site.cfg.example to site.cfg, remove everything and add 
```
[mkl] 
library_dirs = C:\Program Files (x86)\IntelSWTools\sw_dev_tools\compilers_and_libraries\windows\mkl\lib\intel64;C:\Program Files (x86)\IntelSWTools\sw_dev_tools\compilers_and_libraries\windows\compiler\lib\intel64 
include_dirs = C:\Program Files (x86)\IntelSWTools\sw_dev_tools\compilers_and_libraries\windows\mkl\include 
mkl_libs = mkl_rt, libiomp5md
    ```
    Especially the LAST line is important. Using the directive from the example doesn't work. 

*   Build numexpr using 
    
    > python setup.py build 

    Check the output to see whether the MKL is found.  

*   Install numexpr using 

    > python setup.py install 

    Check whether numexpr got compiled with the MKL linked by 
```
import numexpr 
numexpr.get_vml_version() 
```

The main point to link Pythran with MKL when this is the backend of Numpy (which it probably is). In order to link we need to add a .pythranrc file under the user home directory.  
Checkout -> https://pythran.readthedocs.io/en/latest/MANUAL.html#customizing-your-pythranrc  

 
In this file we need to point to the right libraries to included and their locations.  

```
[compiler] 
defines= 
undefs= 
include_dirs=C:\Intel\SDK\compilers_and_libraries\windows\mkl\include 
libs=mkl_rt libiomp5md 
library_dirs=C:\Intel\SDK\compilers_and_libraries\windows\mkl\lib\intel64 C:\Intel\SDK\compilers_and_libraries\\windows\compiler\lib\intel64 
cflags= 
ldflags= 
blas=mkl 
ignoreflags= 
```

*Libiomp5md (and C:\Intel\SDK\compilers_and_libraries\\windows\compiler\lib\intel64) are required for the use of the /openmp directive for multi-threading*

**BUT THERE IS A CATCH**: quite likely the MKL libraries are installed somewhere under 'Program files', but Pythran can't handle paths with spaces (because it takes them as separators instead of ';'). To fix this we can simply place a softlink under a path without spaces which links to some locations. In the example above I placed a softlink to point to the 'sw_dev_tools' directory of the Intel SDK 

> mklink /D "C:\Intel\SDK" "C:\Program Files (x86)\IntelSWTools\sw_dev_tools"  

## Multi-threading
To use multi-threading use the MSVC directive '/openmp' (instead of '-fopenmp' which is for GCC compilers) to link against an openmp library. Make sure to add the openmp library (e.g. libiomp5md) to the .pythranrc file.  

# SIMD support
You may be fooled by thinking that the MSVC compiler does the compiler as the last entry of the build log show the MSVC linker, but the code is in fact compiled by clang (shipped with Pythran). Therefore SIMD support in enabled by 
>  -DUSE_XSIMD -march=native -Ofast /fopenm